# Training a YOLO Model using Google Colab

**Introduction**

I am using this notebook which uses ultralytics to train a YOLOv11 model with a custom dataset.


**Verifing NVIDIA GPU Availability**

Changing the runtime type and selecting GPU under Hardware accelerator. Then running the code below to confirm NVIDIA GPU is available

In [ ]:
!nvidia-smi

**1.&nbsp;Preparing the dataset**

Collecting and labeling the images for training (around 200+ image)

*   Capturing and labeling images

**Labeling Tools:**

*   Label Studio – free and open-source, outputs YOLO format.

**2.1 Uploading The Dataset**

Uploading the `data.zip` file to the Google Colab instance

**2.2 Split images into train and validation folders**

Unzipping the `data.zip` folder and creating some folders to hold the images.

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data

Ultralytics requires a particular folder structure to store training data for models.
Creating the required folder structure by running the following the code.

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

**3.&nbsp;Installing Ultralytics**

Installing the Ultralytics library in this Google Colab instance. This Python library will be used to train the YOLO model.

In [ ]:
!pip install ultralytics

**4.&nbsp;Configure Training**


Running the code below to auto-generate data.yaml.

In [ ]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

**5.&nbsp;Training the Model**

Running the following code block to begin training.

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

The model trains on images in train/ and evaluates on val/ each epoch, reporting mAP, precision, and recall. Training runs for the number of epochs

**6.&nbsp;Testing the Model**

The model has been trained; now it's time to test it! The command below runs the model on the images in the validation folder and then displays the result for the first 10 images.

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


**7.1 Downloading YOLO Model**

Zipping and downloading the trained model by running the code blocks below.

The code creates a folder named `my_model`

In [ ]:
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
from google.colab import files

files.download('/content/my_model.zip')

Using the script `yolo_detect.py` to run the trained model locally.